In [1]:
!pip install PyMuPDF # for reading PDFs with Python
!pip install tqdm
!pip install sentence-transformers # for embedding models
!pip install flash-attn --no-build-isolation
!pip install --upgrade --no-cache-dir gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 37.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [47]:
import torch
import gdown
from sentence_transformers import util
import re
import nltk
from transformers import GPT2TokenizerFast
import fitz
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers.utils import is_flash_attn_2_available
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import textwrap

## Download books

In [5]:
!mkdir books
gdown.download("https://drive.google.com/file/d/1uuKy9ah2VhrjoX7x_k7ccmpz83rt0idg/view?usp=drive_link", "./books/anatomyPhysiology.pdf",fuzzy=True)
# gdown.download("https://drive.google.com/file/d/1K9fzKnEsQhKVl1iVqvpfa8B_qPsasJGd/view?usp=drive_link", "./books/biochemistry.pdf",fuzzy=True)
# gdown.download("https://drive.google.com/file/d/1oBIPvLx0cTVNrROciFXnjMK6EufZt7XN/view?usp=drive_link", "./books/bioPhysiology.pdf",fuzzy=True)
gdown.download("https://drive.google.com/file/d/1ytXII7X3913MATiFj9-kbu8ygi1S8pmf/view?usp=drive_link", "./books/sleep.pdf",fuzzy=True)
gdown.download("https://drive.google.com/file/d/18AsZoeI6VeVHQp8CVgdBFOjNI-0DDOwm/view?usp=drive_link", "./books/humannutrition.pdf",fuzzy=True)

mkdir: cannot create directory ‘books’: File exists


Downloading...
From (original): https://drive.google.com/uc?id=1uuKy9ah2VhrjoX7x_k7ccmpz83rt0idg
From (redirected): https://drive.google.com/uc?id=1uuKy9ah2VhrjoX7x_k7ccmpz83rt0idg&confirm=t&uuid=82370366-8ea6-44ea-aff0-a73827309573
To: /content/books/anatomyPhysiology.pdf
100%|██████████| 452M/452M [00:05<00:00, 77.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ytXII7X3913MATiFj9-kbu8ygi1S8pmf
To: /content/books/sleep.pdf
100%|██████████| 72.3M/72.3M [00:00<00:00, 86.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=18AsZoeI6VeVHQp8CVgdBFOjNI-0DDOwm
To: /content/books/humannutrition.pdf
100%|██████████| 26.9M/26.9M [00:00<00:00, 80.0MB/s]


'./books/humannutrition.pdf'

In [6]:
!ls ./books

anatomyPhysiology.pdf  humannutrition.pdf  sleep.pdf


## Text Formatting

In [7]:
def text_formatter(text: str) -> str:
  """Performs formatting on text."""
  # Replace newlines and multiple spaces with a single space
  text = re.sub(r'\s+', ' ', text)

  # Remove page numbers (assuming they're at the start or end of lines)
  text = re.sub(r'^\s*\d+|\d+\s*$', '', text, flags=re.MULTILINE)

  # Remove common headers/footers (customize as needed)
  text = re.sub(r'^.*Header.*$', '', text, flags=re.MULTILINE)
  text = re.sub(r'^.*Footer.*$', '', text, flags=re.MULTILINE)

  # Strip leading/trailing whitespace
  text = text.strip()

  return text

#splitting text into sentences
nltk.download('punkt')
def get_sentences(text: str) -> int:
  sentences = nltk.sent_tokenize(text)
  return sentences


tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
def count_tokens(text: str) -> int:
  return len(tokenizer.encode(text))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Read in pdfs

In [9]:
def open_and_read_pdf(pdf_path: str, start_page: int = 0) -> list[dict]:
  """
  Opens a PDF file, reads its text content page by page, and collects statistics.

  Parameters:
      pdf_path (str): The file path to the PDF document to be opened and read.
      start_page (int): The page number to start reading from (0-indexed).

  Returns:
      list[dict]: A list of dictionaries, each containing the page number,
      character count, word count, sentence count, token count, and the extracted text
      for each page.
  """
  doc = fitz.open(pdf_path)
  pages_and_texts = []

  for page_number in tqdm(range(start_page, len(doc))):
    page = doc[page_number]
    text = page.get_text()
    cleaned_text = text_formatter(text)
    pageDict = {}
    pageDict["page_number"] = page_number - start_page + 1
    pageDict["page_char_count"] = len(cleaned_text)
    pageDict["page_word_count"] = len(cleaned_text.split())
    pageDict["sentences"] = get_sentences(cleaned_text)
    pageDict["page_sentence_count"] = len(pageDict["sentences"])
    pageDict["page_token_count"] = count_tokens(cleaned_text)
    pageDict["text"] = cleaned_text
    pages_and_texts.append(pageDict)
  return pages_and_texts

In [10]:
pages_and_texts_nutrition = open_and_read_pdf(pdf_path="./books/humannutrition.pdf", start_page=41)
pages_and_texts_sleep = open_and_read_pdf(pdf_path="./books/sleep.pdf", start_page=11)
pages_and_texts_physiology = open_and_read_pdf(pdf_path="./books/anatomyPhysiology.pdf", start_page=23)

100%|██████████| 1294/1294 [00:06<00:00, 215.00it/s]


In [11]:
pages_and_texts = pages_and_texts_nutrition + pages_and_texts_sleep + pages_and_texts_physiology
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,sentences,page_sentence_count,page_token_count,text
0,1,745,117,[Special Thanks to Bill Chismar – University o...,4,159,Special Thanks to Bill Chismar – University of...
1,2,86,15,"[PART I CHAPTER 1., BASIC CONCEPTS IN NUTRITIO...",3,25,PART I CHAPTER 1. BASIC CONCEPTS IN NUTRITION ...
2,3,0,0,[],0,0,
3,4,244,42,[Image by Jim Hollyer / CC BY 4.0 Introduction...,1,76,Image by Jim Hollyer / CC BY 4.0 Introduction ...
4,5,1155,176,[Learning Objectives By the end of this chapte...,10,226,Learning Objectives By the end of this chapter...


### drop blank pages

In [12]:
df = df[df.page_word_count != 0]

### Chunk sentences

In [24]:
num_sentence_chunk = 10
def split_sentences(sentences:list, split_size:int=10) -> list[list[str]]:
  return [sentences[i:i+split_size] for i in range(0,len(sentences), split_size)]
for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_sentences(item["sentences"])
  item["num_chunks"] = len(item["sentence_chunks"])

100%|██████████| 2586/2586 [00:00<00:00, 147800.26it/s]


In [25]:
def process_chunks(pages_text:list[dict]) -> list[dict]:
  pages_chunks = []
  for page_idx, item in enumerate(tqdm(pages_and_texts)):
    for senetence_chunk in item["sentence_chunks"]:
      chunk_dict = {"page_number" : item["page_number"]}

      joined_sentence_chunk = "".join(senetence_chunk).strip()
      joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
      chunk_dict["sentence_chunk"] = joined_sentence_chunk

      chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
      chunk_dict["chunk_word_count"] = len(joined_sentence_chunk.split())
      chunk_dict["chunk_token_count"] = len(tokenizer.encode(joined_sentence_chunk))

      pages_chunks.append(chunk_dict)
  return pages_chunks

In [26]:
pages_and_chunks = process_chunks(pages_and_texts)

100%|██████████| 2586/2586 [00:03<00:00, 737.11it/s]

Total chunks: 5341


In [27]:
print(f"Total chunks: {len(pages_and_chunks)}")

Total chunks: 5341


In [28]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(1)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,5341.0,5341.0,5341.0,5341.0
mean,595.5,981.9,153.7,214.5
std,371.2,664.1,103.8,142.3
min,1.0,5.0,1.0,1.0
25%,266.0,535.0,82.0,119.0
50%,592.0,1009.0,159.0,218.0
75%,918.0,1283.0,203.0,276.0
max,1294.0,4845.0,787.0,1105.0


In [29]:
min_token_length = 30
pages_and_chunks_over_min = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")

### Encoding sentence chunks

In [31]:
sentence_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cuda")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [32]:
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min]

In [33]:
text_chunk_embeddings = sentence_model.encode(text_chunks, batch_size=32, convert_to_tensor=True)

In [34]:
text_chunk_embeddings.shape

torch.Size([5048, 768])

In [35]:
torch.save(text_chunk_embeddings, './books/text_chunk_embeddings.pt')

In [36]:
text_chunk_embeddings = torch.load("./books/text_chunk_embeddings.pt")

<ipython-input-36-9f8d56bcaa98>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_chunk_embeddings = torch.load("./books/text_chunk_embeddings.pt")


In [37]:
text_chunk_embeddings = text_chunk_embeddings.to(torch.float32).to("cuda")

In [38]:
def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [40]:
def retrieve_relevant_embeddings(query:str, embeddings:torch.tensor, model:SentenceTransformer=sentence_model, n_resources:int=5):
  query_embedding = model.encode(query, convert_to_tensor=True)
  dot_product = util.dot_score(a=query_embedding, b=embeddings)[0]
  scores, indices = torch.topk(dot_product, n_resources)
  return scores, indices

In [45]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [48]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="meta-llama/Llama-3.2-1B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [58]:
# quantization_config = BitsAndBytesConfig(load_in_4bit=True)
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="meta-llama/Llama-3.2-1B-Instruct", torch_dtype=torch.float32,
                                                 low_cpu_mem_usage=False,
                                                 attn_implementation="flash_attention_2" if (is_flash_attn_2_available() and torch.cuda.get_device_capability(0)[0] >= 8) else "sdpa", quantization_config=None)

In [59]:
#torch.cuda.empty_cache()
llm_model.to("cuda")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [51]:
def prompt_formatter(query:str, context_items:list)->str:
  context = "- " + "\n- ".join([item for item in context_items])
  base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""

    # Update base prompt with context items and query
  base_prompt = base_prompt.format(context=context, query=query)

    # Create prompt template for instruction-tuned model
  dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]

    # Apply the chat template
  prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
  return prompt

In [52]:
query = "What role does macronutrients play in sleep quality?"
scores, indices = retrieve_relevant_embeddings(query,text_chunk_embeddings, sentence_model)

In [53]:
context_items = [text_chunks[i] for i in indices]

In [54]:
prompt = prompt_formatter(query=query,
                          context_items=context_items)

In [55]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K

In [56]:
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

In [60]:
outputs = llm_model.generate(**input_ids,temperature=0.7,do_sample=True, max_new_tokens=256)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [61]:
output_text = tokenizer.decode(outputs[0])

In [68]:
print(f"Query: {query}")
print("RAG answer")
print_wrapped(output_text.replace(prompt, '').replace("<|begin_of_text|>", '').replace("<|eot_id|>", ''))

Query: What role does macronutrients play in sleep quality?
RAG answer
Macronutrients play a crucial role in sleep quality. The body requires
macronutrients to function properly, and sleep is one of the primary sources of
energy for the body. Adequate macronutrient intake can help promote sleep
quality by:  * Providing the body with the necessary energy for physical
activities * Regulating blood sugar levels * Supporting the release of hormones
that regulate sleep-wake cycles * Affecting the release of neurotransmitters
that promote relaxation and reduce stress  The body's energy production is
primarily based on carbohydrates, which provide a rapid source of energy for the
body. However, excessive carbohydrate intake can lead to an insulin surge, which
can disrupt sleep quality. A diet high in refined sugars and saturated fats can
also lead to insulin resistance, a precursor to type 2 diabetes, which can
further impact sleep quality.  In contrast, macronutrients that promote sleep
qual